# MQ2. 신용거래 이상탐지 데이터 다루기

In [24]:
# Module Import
import pandas as pd 
import numpy as np 
import seaborn as sns

In [25]:
# data load
cc_df = pd.read_csv('/Users/ichangyu/Desktop/ModuLABS/Assignment/251205/fraud.csv')

In [26]:
# data check
pd.set_option('display.max_columns', 50)
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [27]:
cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

In [28]:
cc_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,4.911340e+05,491134.000000,491134.000000,491134.000000,491134.000000,4.911340e+05,4.911340e+05,491134.000000,491134.000000,491134.000000
mean,3.706013e+17,69.050120,50770.532384,37.931230,-90.495619,1.213922e+05,1.358730e+09,37.930272,-90.495411,0.002533
std,1.260229e+18,160.322867,26854.947965,5.341193,12.990732,3.725751e+05,1.819402e+07,5.372986,13.004100,0.050264
min,5.038744e+11,1.000000,1843.000000,24.655700,-122.345600,4.600000e+01,1.325376e+09,23.655789,-123.345106,0.000000
25%,2.131124e+14,8.960000,28405.000000,33.746700,-97.235100,1.228000e+03,1.343087e+09,33.781388,-96.984814,0.000000
50%,3.531130e+15,42.170000,49628.000000,38.507200,-87.591700,5.760000e+03,1.357257e+09,38.545124,-87.573441,0.000000
75%,4.653879e+15,80.330000,75048.000000,41.520500,-80.731000,5.083500e+04,1.374626e+09,41.624294,-80.685567,0.000000
max,4.956829e+18,25086.940000,99323.000000,48.887800,-69.965600,2.906700e+06,1.388534e+09,49.887523,-68.965624,1.000000


## 불필요한 컬럼 제거

In [29]:
# merchant, job, cc_num에 각각 포함된 값이 몇 종류인가?
cc_df['merchant'].nunique() # 693
cc_df['job'].nunique()      # 110
cc_df['cc_num'].nunique()   # 124

# 불필요한 컬럼들 제거
cc_df.drop(['merchant','first','last','street','city',
'state','zip','job','trans_num','unix_time'], axis = 1, inplace= True)

# cc_num 컬럼의 값을 기준으로 정렬
cc_df.sort_values('cc_num')

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
378075,2020-08-05 17:03:19,503874407318,shopping_pos,7.77,M,29.5894,-98.5201,1595797,1975-12-28,30.014539,-98.426767,0
230588,2019-12-20 22:21:36,503874407318,health_fitness,72.06,M,29.5894,-98.5201,1595797,1975-12-28,29.260487,-97.961746,0
421413,2020-10-10 12:39:32,503874407318,misc_pos,4.78,M,29.5894,-98.5201,1595797,1975-12-28,29.116928,-99.258936,0
468378,2020-12-13 15:55:44,503874407318,kids_pets,84.56,M,29.5894,-98.5201,1595797,1975-12-28,29.994537,-98.441297,0
345085,2020-06-22 23:52:06,503874407318,entertainment,24.33,M,29.5894,-98.5201,1595797,1975-12-28,29.277168,-98.239289,0
...,...,...,...,...,...,...,...,...,...,...,...,...
53631,2019-04-14 16:57:31,4956828990005111019,entertainment,27.41,M,40.6747,-74.2239,124967,1980-12-21,39.888460,-74.151440,0
485223,2020-12-27 14:14:40,4956828990005111019,home,28.52,M,40.6747,-74.2239,124967,1980-12-21,41.068207,-75.015316,0
264968,2020-02-15 13:32:48,4956828990005111019,shopping_pos,2.95,M,40.6747,-74.2239,124967,1980-12-21,41.435438,-73.839008,0
63475,2019-04-30 17:45:09,4956828990005111019,shopping_pos,7.28,M,40.6747,-74.2239,124967,1980-12-21,41.149072,-75.143152,0


## 구매금액의 Z-score

In [30]:
# cc_num 컬럼의 값마다 데이터 개수 계산
cc_df['cc_num'].value_counts()

# cc_num 컬럼의 값마다 amt의 평균, 표준편차를 계산하여 amt_info에 저장
amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean','std']).reset_index()

amt_info.to_pickle('./amt_info.pkl')

# cc_num 컬럼을 기준으로, cc_df와 amt_info를 left merge한 cc_df 생성
cc_df = cc_df.merge(amt_info, on='cc_num', how='left')

# 결제금액(amt)의 z-score를 계산해 amt_z 컬럼에 저장
cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

# 사기 거래 데이터(is_fraud = 1)을 확인
cc_df[cc_df['is_fraud'] == 1]   # 1244개

# Z-score 계산 완료 -> mean, std 컬럼 삭제
cc_df.drop(['mean','std'], axis=1, inplace=True)

# cc_num, category 컬럼의 값마다 결제금액(amt)의 평균, 표준편차를 계산하여 cat_info에 저장
cat_info = cc_df.groupby(['cc_num', 'category'])['amt'].agg(['mean','std']).reset_index()

# cat_info를 pickle 파일로 저장
cat_info.to_pickle('./cat_info.pkl')

# cc_num 컬럼을 기준으로, cc_df와 cat_info를 left merge한 cc_df 생성
# 여기서 cc_num 만으로만 merge하면 X
cc_df = cc_df.merge(cat_info, on=['cc_num', 'category'], how='left')

# 결제금액(amt)의 z-score를 계산해 cat_amt_z 컬럼에 저장
cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

# Z-score 계산 완료 -> mean, std 컬럼 삭제
cc_df.drop(['mean','std'], axis =1 , inplace = True)


## 결제 시간 관련 feature 분석

In [31]:
# trans_date_trans_time 컬럼에서 시간 값을 추출해 hour 컬럼 생성
cc_df['hour'] = pd.to_datetime(cc_df['trans_date_trans_time']).dt.hour

In [32]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0


In [33]:
# 결제시간을 morning, afternoon, night, evening으로 분류하기 위한 함수 정의
def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

# hour 컬럼에 hour_func 함수를 적용한 hour_cat 컬럼 생성
cc_df['hour_cat'] = cc_df['hour'].apply(hour_func)        

# 시간대별 데이터 수 계산
cc_df['hour_cat'].value_counts(ascending = True)

# cc_num 컬럼의 값마다 amt 컬럼의 데이터 수를 계산해 all_cnt 변수로 저장
all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index()

# cc_num, hour_cat 컬럼의 값마다 amt 컬럼의 데이터 수를 계산해 hour_cnt 변수로 저장
hour_cnt = cc_df.groupby(['cc_num', 'hour_cat'])['amt'].count().reset_index()

# all_cnt, hour_cnt 값 확인
all_cnt.head()
hour_cnt.head()

# cc_num 컬럼을 기준으로 hour_cnt, all_cnt를 left merge
hour_cnt = hour_cnt.merge(all_cnt, on = 'cc_num',how = 'left')
# hour_cnt.head()

# amt_x, amt_y 컬럼명을 hour_cnt, total_cnt로 변경
hour_cnt = hour_cnt.rename({'amt_x' : 'hour_cnt', 'amt_y' : 'total_cnt'}, axis = 1)
# hour_cnt.head()

# 전체 거래 건수(total_cnt)에 대한 시간대별 거래 건수(hour_cnt)의 비율을 계산해 hour_perc 컬럼에 저장
hour_cnt['hour_perc'] = hour_cnt['hour_cnt'] / hour_cnt['total_cnt']
# hour_cnt.head(10)
# hour_cnt.tail(10)

# cc_num, hour_cat, hour_perc 컬럼만 남기고 저장
hour_cnt = hour_cnt[['cc_num','hour_cat','hour_perc']]
hour_cnt.to_pickle('./hour_cnt.pkl')

# cc_num, hour_cat 컬럼을 기준으로 cc_df와 hour_cnt를 left merge
cc_df = cc_df.merge(hour_cnt, on = ['cc_num', 'hour_cat'], how = 'left')

# 시간 관련 featrue 분석 끝 -> 불필요한 컬럼들 제거
cc_df.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis = 1, inplace = True)

## 거리 관련 feature 분석

In [34]:
# 거리 계산에 필요한 geopy 라이브러리
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [35]:
# distance import : (위도, 경도) 값으로 두 지점 사이 거리 구하기
from geopy.distance import distance

# 고객 위치와 상접 위치 사이 거리를 계산해 distance 컬럼에 저장
# 상당히 오래 걸림
cc_df['distance'] = cc_df.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
# cc_df.head()

# cc_num 컬럼의 값마다 distance의 평균, 표준편차를 계산해 dist_info 변수에 저장
dist_info = cc_df.groupby('cc_num')['distance'].agg(['mean','std']).reset_index()
dist_info.to_pickle('./dist_info.pkl')

# cc_num 컬럼을 기준으로 cc_df와 dist_info를 left merge
# 200만개 이상의 row가 잡힌다면 merge 오류 -> 코드 수정 필요
cc_df = cc_df.merge(dist_info, on = 'cc_num', how = 'left')

# distanca 컬럼의 z-score를 계산해 dist_z 컬럼에 저장
cc_df['dist_z'] = (cc_df['distance'] - cc_df['mean']) / cc_df['std']

# 거리 관련 feature 분석 끝 -> 불필요한 컬럼들 제거
cc_df.drop(['lat', 'long', 'merch_lat', 'merch_long', 'mean', 'std'], axis = 1, inplace = True)

## 나이 feature engineering

In [36]:
# dob 컬럼에서 연도 값만 추출해 dob 컬럼에 저장
cc_df['dob'] = pd.to_datetime(cc_df['dob']).dt.year

## 범주형 데이터 one-hot encoding

In [37]:
# 범주형 컬럼 'category'에 몇 종류의 값이 있는지 확인
cc_df['category'].value_counts() # 14개

# cc_df의 범주형 데이터에 원-핫 인코딩 적용
cc_df = pd.get_dummies(cc_df, columns = ['category'], drop_first = True)
# cc_df.head()

# 이제 불필요해진 cc_num 컬럼 제거
cc_df.drop('cc_num', axis = 1, inplace = True)